# How to set up ROMS/CROCO simulation

Here we will see how to set up ROMS/CROCO files and time for Pyticles. 

Method is based using Modules/R_files.py Classes files and load.

___
## Setting up files

To set up your files you must chose a simulation name and add a switch in 
files Class where you define all files parameters. Here is an example of a
croco simulation named `polgyr`

```Python
# in R_files.py in files()
elif 'polgyr' in simul:
    
    '''
    time 00000 is 2001-01-10 14:53:20
    time 06511 is 2009-12-10 02:53:20
    
    conversion to uncompressed for 2006 = from 3620 to ...
    '''
    
    self.realyear = True
    self.realyear_origin = datetime(1999,1,1)
    self.model = 'croco'
    self.digits = 5

    folder= '/home/datawork-lops-osi/mlecorre/POLGYR/HIS/'
    self.grd='/home/datawork-lops-osi/mlecorre/POLGYR/INIT/polgyr_grd.nc'

    self.his=folder +'polgyr_his.'

    self.frc=folder + '/polgyr_his.03360.nc'
    self.wind=folder + '/polgyr_his.03360.nc'
    self.tfile=20
    self.tstart=0
    self.tend=1000

```
    
Configuration name: 'polgyr'

### model

model used to produce files. It is crucial to determine file names and
and to update file time stepping

- **croco/ucla:** files like `'polgyr_his_03660.nc'`. Note that number of digits can
    be defined by user

- **agrif:**
  ROMS AGRIF files
  - **CLIMATOLOGY**: naming for climatology runs with one file per month and 
  a fixed number of outputs per file (usually 30 per month).
  `roms_his_Y2000M1.nc`
  `self.realyear = False` in R_file.py, Class files()
  - **INTERANNUAL:** naming for interannual files with varying number of
  days per month. Also `roms_his_Y2000M1.nc` but 31 days in this one.

- **croco_xios** CROCO naming with XIOS CPP key:  `croco_1999-01-25-1999-01-29.nc`

### file naming and location

- **his** is used as a prefix for file naming

- **grid** should point to grid file (or file with grid parameters)

- **frc** and **wind** are optionnal

- **fileformat**: prefix of the file `.nc` most likely

### metadata parameters

- **realyear** is True for real calendar, otherwise 360 days and 30 days per month

- **realyear_origin**: datetime used for t=0 in croco simulation.
  This is also the date used to load at __init__ method of load()
  Class from R_files.py

- **realyear_tstart**: date that matched t_start=0 for Pyticles.
  This is very usefull when spinup is removed on disk to save some space.
  Typicall case is to remove a few years, then origin date and start date
  are different. 

- **digits**: number of digits to name files (2: 00, 3: 000, ...)

- **dtfile**: time step of ROMS/CROCO file in seconds

- **tfile** number of time steps per file (when fixed)

- **tstart**: time step within CROCO/ROMS file to start from. Usually it is
  set to zero, but it is possible to start from any time step between 0 and
  the number of time steps within the first file 

- **tend**: last time of simulation, take it big.

Note that it is possible to use analytical fields inputs.
___
## Examples

This section present some parameters for each velocities inputs type

```Python
if 'ucla' in simul:
    folder = '/ucla'
    self.his = folder + '/roms_'
    self.grd = folder + '/roms_grd.nc'
    self.frc = folder + '/roms_frc.nc'
    self.wind = folder + '/roms_frc.nc'
    self.tfile = 5
    self.tstart = 0
    self.tend = 1000

# croco interannuel 'roms_his_Y20M3.nc'
elif 'croco' in simul:
    if 'date_inter' in simul:
        self.model = 'croco_date'
        self.realyear = True
        self.realyear_origin = datetime(2007, 1, 1)
        folder= '/data/croco_inter'
        self.his = folder + '/roms_his_'
        self.grd = folder + '/roms_grid.nc'
        self.frc = self.grd
        self.wind = self.grd
        self.tstart = 0
        self.tend = 1000

    # croco climatology 'roms_his_Y20M3.nc'
    elif 'date_clim' in simul:
        self.model = 'croco_date'
        self.realyear = False
        folder = '/croco_clim'
        self.his = folder + '/roms_his_'
        self.grd = folder + '/roms_his_Y20M1.nc'
        self.frc = folder + '/roms_his_Y20M1.nc'
        self.wind = folder + '/roms_his_Y20M1.nc'
        self.tfile = 30
        self.tstart = 0
        self.Ystart = 20
        self.Mstart = 1
        self.tend = 1000

    # croco xios 'croco_his_2001-01-06-2001-01-10.nc'
    elif 'xios' in simul:
        self.realyear = True
        self.realyear_origin = datetime(1999, 1, 1)
        self.realyear_tstart = datetime(2001, 1, 1)
        self.model = 'croco_xios'

        folder = '/croco_xios'
        self.grd = folder + '/grd.nc'

        self.his = folder + '/croco_his_'
        self.dtfile = 12 * 3600
        self.tfile = 10
        self.tstart = 0
        self.tend = 120    

    else:
        # default croco croco_his_1000.nc
        self.realyear = True
        self.realyear_origin = datetime(1999, 1, 1)
        self.model = 'croco'
        self.digits = 5

        folder = '/croco'
        self.grd = folder +'/grd.nc'
        self.his = folder +'/polgyr_his.'
        self.frc = folder + '/polgyr_his.03360.nc'
        self.wind = folder + '/polgyr_his.03360.nc'
        self.tfile = 20
        self.tstart = 0
        self.tend = 1000
```

___
## practical case

It is a good practise before running to Pyticles to ensure that files are
correctly named and that time stepping is alright.

if files are not present on disk it is possibile to use init and update methods
from Class load using parameters touchfile = False, loadgrid = False

In [87]:
import sys
import numpy as np
from importlib import reload

# pyticles related modules and tools
sys.path.append("../../Modules/")
import R_files
from R_files import load
reload(R_files)
from R_files import load

here we load CROCO files from Polgyr simulation (available on Datarmor)

output logs should give you some information about simulation files and variables.

In [134]:
my_simul = 'polgyr'
simul = load(my_simul, touchfile=False)

simulname is polgyr
args (['polgyr'],)
args[0] ['polgyr']
len(args[0]) 1
/scratch/Jcollin/croco/polgyr/polgyr_his.00000.nc
not touching _his file, loading _grd anyway
[0, -1, 0, -1, [0]]
ncname0,ny1,ny2,nx1,nx2
/scratch/Jcollin/croco/polgyr/polgyr_grd.nc 0 -1 0 -1
[topo,pm,pn,f,lat,lon] have just been loaded
----------------------------------------------------------
All arrays are now Fortran ordered and indices are [i,j,k]
----------------------------------------------------------
(2002, 1602)


To check that time stepping is alright

- time 00000 is 2001-01-10 14:53:20

- and time 06511 is 2009-12-10 02:53:20

Using load method, and filetime, infiletime attributes

**NOTE** This only works when file is on disk

In [133]:
reload(R_files)

from R_files import test
test()

Start to test R_files.py methods

start update 

self.time=59
self.ncfile='/data/croco_inter/roms_his_Y2007M3.nc'
self.infiletime=0
code_test_croco_date_inter: OK

self.time=60
self.ncfile='/croco_clim/roms_his_Y20M3.nc'
self.infiletime=0
code_test_croco_date_clim: OK

self.time=10
self.ncfile='/croco_xios/croco_his_2001-01-06-2001-01-10.nc'
self.infiletime=0
code_test_croco_xios: OK

self.time=145
self.ncfile='/croco/polgyr_his.00140.nc'
self.infiletime=5
code_test_croco: OK

self.time=12
self.ncfile='/ucla/roms_0010.nc'
self.infiletime=2
code_test_ucla: OK

self.time=1
self.ncfile='/home/gula/libra/ROMS/Simulations/Rutgers_example/velocity_example.nc'
self.infiletime=1
code_test_rutgers: OK

update ok
------------------
test OK

